# --- Day 9: Movie Theater ---
https://adventofcode.com/2025/day/9

In [ ]:
import matplotlib as plt

def getTestInput():
	with open("sampleInput.txt") as file:
		return file.read()
	
def getInput():
	with open("redTiles.txt") as file:
		return file.read()

In [61]:
redTiles = getInput().splitlines()

def getArea(corner1: str, corner2: str) -> int:
	"""Gets the area of a rectangle given two opposite corners"""
	x1, y1 = [int(x) for x in corner1.split(",")]
	x2, y2 = [int(x) for x in corner2.split(",")]
	return abs(x1-x2+1) * abs(y1-y2+1)

# Loop through each corner pair
areas = []
for i, corner1 in enumerate(redTiles):
	for j, corner2 in enumerate(redTiles[i+1:]):
		if i >= j: continue
		areas.append(getArea(corner1, corner2))

print(f"Largest rectangle area: {max(areas)}")

Largest rectangle area: 4782896435


# --- Part Two ---

In [92]:
redTiles = getInput().splitlines()
#redTiles = getTestInput().splitlines()
redTiles = [tuple(map(int, x.split(","))) for x in redTiles]
redTiles.append(redTiles[0]) # Make sure we loop around

def printFloorPlan(redTiles, borders, outerBorder, xMax, yMax):
    """TEST OUTPUT"""
    for x in range(xMax):
        for y in range(yMax):
            if (y,x) in redTiles:
                print("#", end="")
            elif (y,x) in borders:
                print(".", end="")
            elif (y,x) in outerBorder:
                print("@", end="")
            else:
                print(".", end="")
        print()
        
def getArea(corner1: str, corner2: str) -> int:
	"""Gets the area of a rectangle given two opposite corners"""
	x1, y1 = [int(x) for x in corner1.split(",")]
	x2, y2 = [int(x) for x in corner2.split(",")]
	return abs(x1-x2+1) * abs(y1-y2+1)

def getNextBorder(tile1: list[int, int], tile2: list[int, int]) -> list[list[int, int]]:
    """Get a border from 2 tiles"""
    x1, y1 = tile1
    x2, y2 = tile2
    if x1 == x2:
        return set([(x1, y) for y in range(min(y1, y2), max(y1,y2)+1)])
    return set([(x, y1) for x in range(min(x1, x2), max(x1,x2)+1)])

def getPossibleOuterBorderStarts(tile1: list[int, int], tile2: list[int, int]) -> tuple[list[int, int], list[int, int], str]:
    """ 
    Takes in two tiles that are connected and returns an inner border and outer border start coordinate e.g.
    ....O.....
    ..XXXXX...
    ....I.....
    Finds the middle of the border and gets coords on either side (does not know which is which)
    """
    x1, y1 = tile1
    x2, y2 = tile2
    xMid = (x1 + x2) // 2
    yMid = (y1 + y2) // 2
    if x1 == x2:
        return [x1 + 1, yMid], [x1 - 1, yMid], "right"
    return [xMid, y1 + 1], [xMid, y1 - 1], "up"

def getOuterBorder(x: int, y: int, direction: str) -> set[tuple[int, int]]:
    """Find outer border using dfs.
    Must start moving clockwise"""
    # If we've already visited this one, return explored
    stack = [(x, y, direction)]
    explored = set()

    while stack:
        x, y, direction = stack.pop()

        # If we hit a border then that means it's the inner boundary, so return an empty set
        if (x,y) in borders:
            return set()

        right = (x,y+1)
        left = (x,y-1)
        up = (x-1,y)
        down = (x+1,y)
        
        # If moving right and blocked right, move up
        if direction == "right" and right in borders:
            if up not in explored:
                stack.append((up[0], up[1], "up"))
                explored.add(up)
        # If moving right and nothing below, move down
        elif direction == "right" and down not in borders:
            if down not in explored:
                stack.append((down[0], down[1], "down"))
                explored.add(down)
        # If moving right and no blockers, just move right
        elif direction == "right":
            if right not in explored:
                stack.append((right[0], right[1], "right"))
                explored.add(right)

        # If moving left and blocked left, move down
        elif direction == "left" and left in borders:
            if down not in explored:
                stack.append((down[0], down[1], "down"))
                explored.add(down)
        # If moving left and nothing above, move up
        elif direction == "left" and up not in borders:
            if up not in explored:
                stack.append((up[0], up[1], "up"))
                explored.add(up)
        # If moving left and no blockers, just move left
        elif direction == "left":
            if left not in explored:
                stack.append((left[0], left[1], "left"))
                explored.add(left)

        # If moving up and blocked up, move left
        elif direction == "up" and up in borders:
            if left not in explored:
                stack.append((left[0], left[1], "left"))
                explored.add(left)
        # If moving up and nothing right, move right
        elif direction == "up" and right not in borders:
            if right not in explored:
                stack.append((right[0], right[1], "right"))
                explored.add(right)
        # If moving up and no blockers, just move up
        elif direction == "up":
            if up not in explored:
                stack.append((up[0], up[1], "up"))
                explored.add(up)

        # If moving down and blocked down, move right
        elif direction == "down" and down in borders:
            if right not in explored:
                stack.append((right[0], right[1], "right"))
                explored.add(right)
        # If moving down and nothing left, move left
        elif direction == "down" and left not in borders:
            if left not in explored:
                stack.append((left[0], left[1], "left"))
                explored.add(left)
        # If moving down and no blockers, just move down
        elif direction == "down":
            if down not in explored:
                stack.append((down[0], down[1], "down"))
                explored.add(down)
    
    return explored

borders = set()
for i in range(len(redTiles) - 1):
    borders |= getNextBorder(redTiles[i], redTiles[i + 1])

possibleOuter1, possibleOuter2, direction = getPossibleOuterBorderStarts(redTiles[0],redTiles[1])
print(possibleOuter1, possibleOuter2, direction)
print(f"Start point: {possibleOuter1}, Direction: {direction}, Border length: {len(getOuterBorder(possibleOuter1[0], possibleOuter1[1], direction))}")
#printFloorPlan(redTiles, borders, getOuterBorder(*possibleOuter1, direction), 9, 14) # TEST
print(f"Start point: {possibleOuter2}, Direction: {direction}, Border length: {len(getOuterBorder(possibleOuter2[0], possibleOuter2[1], direction))}")
#printFloorPlan(redTiles, borders, getOuterBorder(*possibleOuter2, direction), 9, 14) # TEST

[97669, 50771] [97667, 50771] right
Start point: [97669, 50771], Direction: right, Border length: 595920
Start point: [97667, 50771], Direction: right, Border length: 595902
